# Общие эпитопы ковида и человека
### Ищем регионы шириной 9 и 14 а.к., сходство минимум 90%
***

# [18.09]

In [37]:
peplen = 7

In [38]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

batch = 20
peps = []
with open("wuhan_hu_1.faa", "r") as handle:
    q = 1
    for record in SeqIO.parse(handle, "fasta"):
        for i in range(1, len(record.seq)-peplen):
            seq_name = record.id + "_" + str(i)
            seq = Seq(str(record.seq[i:i+peplen]))
            sr = SeqRecord(seq, seq_name, '', '')
            peps.append(sr)
            if(len(peps)>batch):
                fname="peptides/batch_"+str(q)+".fasta"
                with open(fname, "w") as output_handle:
                    SeqIO.write(peps, output_handle, "fasta")
                    q = q + 1
                    peps = []

In [ ]:
!blastp -db uniprot_human -query peptides/batch_84.fasta -word_size 3 -evalue 1000 -outfmt '6 std nident salltitles'

In [ ]:
!snakemake --cores 120 -rk

In [46]:
!mkdir -p result
!cat blast/*.out > result/blast.out 

In [51]:
print(peplen)
import pandas as pd
blast = pd.read_csv("result/blast.out", sep="\t", header = None)
blast.columns=["qseqid", 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'nident','sname']
blast
    

7


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,nident,sname
0,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A096LP60|A0A096LP60_HUMAN,85.714,7,1,0,1,7,75,81,59.0,18.1,6,tr|A0A096LP60|A0A096LP60_HUMAN AP2-associated ...
1,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A096LP25|A0A096LP25_HUMAN,85.714,7,1,0,1,7,81,87,65.0,17.7,6,tr|A0A096LP25|A0A096LP25_HUMAN AP2-associated ...
2,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A096LNZ0|A0A096LNZ0_HUMAN,85.714,7,1,0,1,7,81,87,69.0,17.7,6,tr|A0A096LNZ0|A0A096LNZ0_HUMAN AP2-associated ...
3,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|H0Y7G5|H0Y7G5_HUMAN,83.333,6,1,0,2,7,81,86,32.0,18.9,5,tr|H0Y7G5|H0Y7G5_HUMAN 85/88 kDa calcium-indep...
4,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A590UK51|A0A590UK51_HUMAN,83.333,6,1,0,2,7,250,255,40.0,18.5,5,tr|A0A590UK51|A0A590UK51_HUMAN 85/88 kDa calci...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11097,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|U3KQ44|U3KQ44_HUMAN,85.714,7,1,0,1,7,46,52,90.0,17.7,6,tr|U3KQ44|U3KQ44_HUMAN TM2 domain-containing p...
11098,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|U3KQF9|U3KQF9_HUMAN,85.714,7,1,0,1,7,23,29,90.0,17.7,6,tr|U3KQF9|U3KQF9_HUMAN TM2 domain-containing p...
11099,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|J3QL70|J3QL70_HUMAN,100.000,6,0,0,1,6,334,339,96.0,17.3,6,tr|J3QL70|J3QL70_HUMAN Regulator of G-protein-...
11100,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|F8VPD4|F8VPD4_HUMAN,71.429,7,2,0,1,7,1149,1155,89.0,17.3,5,tr|F8VPD4|F8VPD4_HUMAN Aspartate carbamoyltran...


In [52]:
blast_ok = blast[blast["nident"]==peplen]
blast_ok.to_csv("result/blast_filtered_7.out", sep="\t")
blast_ok


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,nident,sname
57,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A0A0MTS7|A0A0A0MTS7_HUMAN,100.0,7,0,0,1,7,9097,9103,23.0,19.2,7,tr|A0A0A0MTS7|A0A0A0MTS7_HUMAN Titin OS=Homo s...
62,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|E9PL21|E9PL21_HUMAN,100.0,7,0,0,1,7,4,10,15.0,19.2,7,tr|E9PL21|E9PL21_HUMAN Elongation factor 1-del...
63,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|E9PRY8|E9PRY8_HUMAN,100.0,7,0,0,1,7,54,60,21.0,19.2,7,tr|E9PRY8|E9PRY8_HUMAN Elongation factor 1-del...
64,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A087X1X7|A0A087X1X7_HUMAN,100.0,7,0,0,1,7,4,10,21.0,19.2,7,tr|A0A087X1X7|A0A087X1X7_HUMAN Elongation fact...
65,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|E9PQR8|E9PQR8_HUMAN,100.0,7,0,0,1,7,4,10,22.0,19.2,7,tr|E9PQR8|E9PQR8_HUMAN Elongation factor 1-del...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10329,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|I3NI20|I3NI20_HUMAN,100.0,7,0,0,1,7,151,157,21.0,19.2,7,tr|I3NI20|I3NI20_HUMAN Adenylate cyclase type ...
10352,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|Q5JV61|Q5JV61_HUMAN,100.0,7,0,0,1,7,36,42,47.0,18.5,7,tr|Q5JV61|Q5JV61_HUMAN FERM domain-containing ...
11076,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A1B0GTM5|A0A1B0GTM5_HUMAN,100.0,7,0,0,1,7,99,105,85.0,17.7,7,tr|A0A1B0GTM5|A0A1B0GTM5_HUMAN Leucine-rich gl...
11077,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A0D9SFU4|A0A0D9SFU4_HUMAN,100.0,7,0,0,1,7,75,81,99.0,17.3,7,tr|A0A0D9SFU4|A0A0D9SFU4_HUMAN Leucine-rich gl...


# [16.09]

In [3]:
gunzip sequences_2020-06-16_13-19.fasta.gz

In [6]:
makeblastdb -in sequences_2020-06-16_13-19.fasta -out uniprot_human -dbtype prot



Building a new DB, current time: 09/16/2020 16:03:22
New DB name:   /data11/bio/PROJECTS/CoV/commotSeqsWithHuman/uniprot_human
New DB title:  sequences_2020-06-16_13-19.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 54715 sequences in 1.02822 seconds.


In [5]:
cp ../metatranscriptome/ref/wuhan_hu_1.gb .

In [2]:
#faa_sars="../gisaid/gisaid/prokka/hCoV_19_Wuhan_Hu_1/hCoV_19_Wuhan_Hu_1.faa"

In [8]:
GB2faa.pl wuhan_hu_1.gb > wuhan_hu_1.faa
faa_sars="wuhan_hu_1.faa"

In [15]:
blastp -db uniprot_human -query $faa_sars -outfmt 6 -evalue 30 > blast_results
wc -l blast_results

143 blast_results


In [10]:
blastp -db uniprot_human -query $faa_sars > blast_results_full

### look blast results
***

In [4]:
blast = read.delim("blast_results", head = F)

In [6]:
colnames(blast) = c("qid", "sid", "identity", "length", "mismatches", "gaps", "qstart", "qend", "sstart", "send", "evalue", "bitscore")

In [10]:
blast = blast[order(-blast$identity),]


In [11]:
head(blast)

,qid,sid,identity,length,mismatches,gaps,qstart,qend,sstart,send,evalue,bitscore
14,hCoV_19_Wuhan_Hu_1_00002,tr|A0A5K1VW56|A0A5K1VW56_HUMAN,53.846,26,12,0,1096,1121,237,262,3.2,32.7
44,hCoV_19_Wuhan_Hu_1_00004,tr|A0A0A0MT16|A0A0A0MT16_HUMAN,50.000,20,10,0,127,146,12,31,3.4,29.6
50,hCoV_19_Wuhan_Hu_1_00007,tr|K7EQ58|K7EQ58_HUMAN,47.368,19,10,0,35,53,116,134,3.7,26.6
51,hCoV_19_Wuhan_Hu_1_00007,tr|E9PL27|E9PL27_HUMAN,47.368,19,10,0,35,53,116,134,3.8,26.6
52,hCoV_19_Wuhan_Hu_1_00007,tr|E9PRR6|E9PRR6_HUMAN,47.368,19,10,0,35,53,116,134,4.5,26.6
57,hCoV_19_Wuhan_Hu_1_00008,tr|B4DWB8|B4DWB8_HUMAN,44.828,29,15,1,76,103,140,168,2.3,27.7
